In [4]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


import the csv spotify dataset using pandas


In [5]:
import pandas as pd

# Load the data
df = pd.read_csv('kaggle_spotify_training_set/spotify_millsongdata.csv')

#we will get the shape, display the dataframe(first items(head)) and see if there is any null values.
df.shape
df.head()
#df.isnull().sum()



,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


we will drop the link column as it is not useful for our analysis coze we only need the artist, song and text(lyrics)


In [6]:
#we will drop the link column as it is not useful for our analysis coze we only need the artist, song and text(lyrics)
df = df.drop(['link'], axis=1).reset_index(drop=True)

we will only use like 5000 samples not 57thousand+.


In [7]:
df = df.sample(5000)
df.shape

(5000, 3)

we will ow clean up the lyrics(text), by removing the regexs and making the texts small letters


In [8]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ', regex=True).replace(r'\n', ' ', regex=True)
df.head()

,artist,song,text
42729,Mariah Carey,The One,see i'm going through a situation \r that i c...
38622,Joni Mitchell,Overture,red sun came rolling down a grey sky \r and ...
55999,Westlife,Lighthouse,this kind of love \r is more than a lifeline ...
3623,Dan Fogelberg,Over And Over,well i got a lover and i got a friend \r she ...
27147,Chaka Khan,Twisted,"here, certified unsound i feel like you \r th..."


we will now tokenize out lyrics(texts). like we will make words that are similar like beautiful, beautifully, beauty into one single token them stem them.so we will use nltk and nltk stem porter


In [9]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
#nltk.download()
from nltk.stem import PorterStemmer

# Download necessary NLTK data
nltk.download('punkt')

# Create a stemmer instance
stemmer = PorterStemmer()

# Define the tokenize method
def tokenize(txt):
    tokens = nltk.word_tokenize(txt)
    # Use the stemmer to stem each token
    stemmed = [stemmer.stem(t) for t in tokens]
    return " ".join(stemmed)

# Test the tokenize method
print(tokenize("I love your beautiful, beauty and lovely face thats full of love"))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


i love your beauti , beauti and love face that full of love


we will now use Tfidvectorizer to verctorize the lyrics, then use cosine_similarities, both using sklearn to get the songs that are similar through their cosine distance and similarity after the lyrics are vectorized.


In [11]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


we will create a lambda function to apply the tokenize method to the text column

In [14]:
df['text'].apply(lambda x: tokenize(x))

42729    see i 'm go through a situat that i ca n't hel...
38622    red sun came roll down a grey sky and the frog...
55999    thi kind of love is more than a lifelin for a ...
3623     well i got a lover and i got a friend she say ...
27147    here , certifi unsound i feel like you there '...
                               ...                        
55549    hooray for the last of the mad hooray for no m...
11450    i 'm forty-on , the moon is full , you make lo...
4891     sometim i think i 'm go crazi stare at the sam...
31382    oh ! it 's music to my ear , oh man ! how can ...
49893    rebel , rebel and yell 'cau our peopl still dw...
Name: text, Length: 5000, dtype: object

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

we will now anaylize them to make sure they are words and set a stop if its not english, then transform the text(lyrics) into a matrix, which we will pass to the cosine_similarity.

In [18]:
# Create a TfidfVectorizer instance
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

# Transform the text (lyrics) into a matrix
lyrics_matrix = vectorizer.fit_transform(df['text'])

# Calculate the cosine similarity between the lyrics
cosine_sim = cosine_similarity(lyrics_matrix)
cosine_sim[0]

array([1.        , 0.01596858, 0.08386323, ..., 0.06043052, 0.06957651,
       0.01419556])